In [2]:
import random
import time

import pygame
import os

random_seed = 42
random.seed(random_seed)

pygame 2.5.2 (SDL 2.28.5, Python 3.11.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


### Reading Input

In [3]:
file_path = "dataset/test/4.txt"
with open(file_path) as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

labyrinth = lines[:-2]
start_coords = lines[-2]
end_coords = lines[-1]

print("\n".join(labyrinth), start_coords, end_coords, sep='\n')

XXXXX
X   X
X   X
X X X
XXXXX
start 3, 3
end 1, 1


In [4]:
def get_coords(coords_string):
    split = coords_string.split()
    x = split[1].split(sep=',')[0]
    y = split[2]
    return int(x), int(y)

In [5]:
start = get_coords(start_coords)
end = get_coords(end_coords)

print(f"start: {start}", f"end: {end}", sep='\n')

start: (3, 3)
end: (1, 1)


### Construction of the graph

In [6]:
labyrinth

['XXXXX', 'X   X', 'X   X', 'X X X', 'XXXXX']

In [7]:
labyrinth_width = len(labyrinth[0])
labyrinth_height = len(labyrinth)

In [8]:
graph = {}
walls = []
for y in range(len(labyrinth)):
    for x in range(len(labyrinth[0])):
        if labyrinth[y][x] == "X":
            walls.append((x, y))
            continue
        edges = []
        for delta_x, delta_y in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            neigh_x, neigh_y = x + delta_x, y + delta_y
            if labyrinth[neigh_y][neigh_x] == " ":
                edges.append((neigh_x, neigh_y))
        graph[(x, y)] = edges

print(f"Walls: {walls}")
print(f"Graph: {graph}")


Walls: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (0, 1), (4, 1), (0, 2), (4, 2), (0, 3), (2, 3), (4, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
Graph: {(1, 1): [(2, 1), (1, 2)], (2, 1): [(3, 1), (1, 1), (2, 2)], (3, 1): [(2, 1), (3, 2)], (1, 2): [(2, 2), (1, 3), (1, 1)], (2, 2): [(3, 2), (1, 2), (2, 1)], (3, 2): [(2, 2), (3, 3), (3, 1)], (1, 3): [(1, 2)], (3, 3): [(3, 2)]}


# Random Search

In [9]:
def random_search(graph, start, end):
    predecessors = dict()
    opened = set()
    opened.add(start)
    expanded = set()
    while len(opened) != 0:
        current = random.choice(list(opened))
        if current == end:
            return predecessors, len(expanded)
        if current in graph:
            for neighbor in graph[current]:
                if neighbor not in opened | expanded:
                    opened.add(neighbor)
                    predecessors[neighbor] = current
        opened.remove(current)
        expanded.add(current)


def reconstruct_path(predecessors, end):
    path = []
    pred = end
    while pred is not None:
        path.append(pred)
        if pred not in predecessors:
            pred = None
        else:
            pred = predecessors[pred]
    path.reverse()
    return path


result = random_search(graph, start, end)
predecessors, expanded = result
path = reconstruct_path(predecessors, end)

print("Predecessors:", predecessors)
print("Shortest path: ", path)
print("Shortest path len: ", len(path) - 1)
print("Expanded: ", expanded)


Predecessors: {(3, 2): (3, 3), (2, 2): (3, 2), (3, 1): (3, 2), (1, 2): (2, 2), (2, 1): (2, 2), (1, 3): (1, 2), (1, 1): (1, 2)}
Shortest path:  [(3, 3), (3, 2), (2, 2), (1, 2), (1, 1)]
Shortest path len:  4
Expanded:  6


### Setting Up Pygame For Visualization

In [10]:
CELL_SIZE = 40
WALL_COLOR = (0, 0, 0)
STEP_INTERVAL = 1000  # milliseconds

In [13]:
pygame.init()
screen = pygame.display.set_mode((labyrinth_width * CELL_SIZE, labyrinth_height * CELL_SIZE))
pygame.display.set_caption("Random Search")

running = True

screen.fill("white")
pygame.display.flip()

while running:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    for wall in walls:
        time.sleep(1)
        x, y = wall
        pygame.draw.rect(surface=screen, color=WALL_COLOR,
                         rect=(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
        pygame.display.flip()

pygame.quit()

# BFS

# DFS

# Greedy Search

# A*